In [1]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *
import hashlib

In [2]:
# Ruta credenciales de CGP
key_path = key_path

# Datos de origen de datos en Bigquery
project = project_id
dataset = 'bronze'
table = 'bronze_ticker_data'
table_conca = f'{project}.{dataset}.{table}'

In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [4]:
# Nos traemos los ticker de las empresas del S&P500
unique_tickers = bigquery.run_query(
    f"""
    SELECT DISTINCT
        inf.ticker
    FROM sara-carles-keepcoding.bronze.bronze_ticker_info AS inf
    INNER JOIN `bronze.bronze_sp500_tickers` AS sp
    ON sp.ticker = inf.ticker
    """
)
unique_tickers

,ticker
0,ODFL
1,REGN
2,VRTX
3,DVN
4,ROST
...,...
498,NRG
499,VST
500,VTRS
501,ZTS


In [5]:
# Asigamos una fecha inicial a cada ticker
unique_tickers_initial_date = initial_date_by_ticker(unique_tickers, initial_date='2015-01-01')

In [6]:
# Attempt to fetch the maximum date per ticker from BigQuery; fallback to initial dates if an error occurs.
try:
    max_date_by_ticker = bigquery.run_query(
        f"""
        SELECT
            ticker,
            max(date) as date 
        FROM {table_conca} 
        GROUP BY 
            ticker
        """
    )
    max_date_by_ticker['date'] = pd.to_datetime(max_date_by_ticker['date'])
    max_date_by_ticker['date'] = max_date_by_ticker['date'].dt.tz_localize(None)
    
    # Merge with the DataFrame of unique tickers
    max_date_by_ticker = pd.merge(unique_tickers_initial_date, max_date_by_ticker, how='left', on='ticker')

    # Keep the maximum date between both fields
    max_date_by_ticker['date'] = max_date_by_ticker[['date', 'initial_date']].max(axis=1)

    # Remove the auxiliary 'initial_date' column
    max_date_by_ticker.drop(columns=['initial_date'], inplace=True)

except Exception as e:
    print(e)
    max_date_by_ticker = unique_tickers_initial_date
    print(max_date_by_ticker)

In [7]:
# Intentar obtener datos históricos con una columna de fecha inicial
try:
    new_df = fetch_historical_data(max_date_by_ticker, start_date_col='date', interval='1d')
except Exception as e:
    print(f"Error al obtener datos con 'date': {e}")
    new_df = fetch_historical_data(max_date_by_ticker, start_date_col='initial_date', interval='1d')
    print("Datos obtenidos con 'initial_date'.")

# Mostrar el DataFrame obtenido
print(new_df)

Datos obtenidos para ODFL.
Datos obtenidos para REGN.
Datos obtenidos para VRTX.
Datos obtenidos para DVN.
Datos obtenidos para ROST.
Datos obtenidos para AVGO.
Datos obtenidos para QCOM.
Datos obtenidos para DG.
Datos obtenidos para DLTR.
Datos obtenidos para BIO.
Datos obtenidos para CCL.
Datos obtenidos para PFG.
Datos obtenidos para TROW.
Datos obtenidos para PRU.
Datos obtenidos para SMCI.
Datos obtenidos para NTAP.
Datos obtenidos para WTW.
Datos obtenidos para AXON.
Datos obtenidos para VRSK.
Datos obtenidos para AAPL.
Datos obtenidos para HSIC.
Datos obtenidos para WAT.
Datos obtenidos para IDXX.
Datos obtenidos para PAYX.
Datos obtenidos para ZBRA.
Datos obtenidos para CSCO.
Datos obtenidos para POOL.
Datos obtenidos para FAST.
Datos obtenidos para FTNT.
Datos obtenidos para ADBE.
Datos obtenidos para MSFT.
Datos obtenidos para ORCL.
Datos obtenidos para CTAS.
Datos obtenidos para AWK.
Datos obtenidos para AMGN.
Datos obtenidos para DOC.
Datos obtenidos para EL.
Datos obtenido

In [8]:
# Fields that make up the ID
id_fields = ['ticker', 'date']

# Apply the function to the DataFrame to create the 'id' column
new_df['id'] = new_df.apply(generate_id, axis=1, fields=id_fields)

new_df

,date,open,high,low,close,adjclose,volume,ticker,id
0,2024-08-27,3.630000,3.750000,3.480000,3.710000,3.710000,1595600.0,EU,b1fdb5052a63df376b4fd7e0cf383ad1
1,2024-08-28,3.630000,3.680000,3.550000,3.550000,3.550000,1395100.0,EU,8c5ea9c76f399e00858a72d59046c48e
2,2024-08-29,3.550000,3.580000,3.340000,3.470000,3.470000,2070800.0,EU,f350be0f12c3cf6247929cd9c46a3e27
3,2024-08-27,38.599998,39.750000,38.119999,39.470001,39.470001,157100.0,LEU,ca83797edf21abc064d68ece6af6d936
4,2024-08-28,39.270000,39.299999,37.730000,38.000000,38.000000,171200.0,LEU,52e6453ae20c48ad1092d264924046e9
...,...,...,...,...,...,...,...,...,...
3086171,2024-08-23,4.350000,4.460000,4.310000,4.410000,4.410000,4323100.0,SBSW,67ef4ea057fbfe9f518b222f1c888adc
3086172,2024-08-26,4.470000,4.490000,4.370000,4.400000,4.400000,2687800.0,SBSW,4c1ce088b295bd9538533abde2b6070f
3086173,2024-08-27,4.390000,4.440000,4.350000,4.400000,4.400000,1907200.0,SBSW,b93f66cca869581a5a4e190fa5ba0f81
3086174,2024-08-28,4.250000,4.260000,4.080000,4.120000,4.120000,4590700.0,SBSW,214c4b20855d548a89899aaacf29d278


In [10]:
try:
    # Filtramos solamente los nuevos registros
    df_incremental = bigquery.select_for_incremental(id='id', table=table_conca, new_df=new_df)

    if not df_incremental.empty:
        # Guardamos los datos en Bigquery
        bigquery.save_dataframe(df_incremental, project, dataset, table, if_exists='append', schema=None)
        print(f'New records loaded')
    else:
        print('No new records to load.')

# En el caso de no tener datos en Bigquery, guardamos todo el df
except Exception as e:
    # bigquery.save_dataframe(new_df, project, dataset, table, if_exists='replace', schema=None)
    # print('New data persisted')
    print(f'Exception encountered: {e}')

100%|██████████| 1/1 [00:00<00:00, 990.16it/s]

New records loaded
